In [4]:
from mediaeval_fake_news.prepare_graphs_basic_info import get_training_dataframe
from mediaeval_fake_news.subgraph_sampling import subgraph_random_sampling
from mediaeval_fake_news.graph2vec import get_weighted_average_embeddings, get_largest_subgraph_embedding
from mediaeval_fake_news.deepwalk import clean_for_deepwalk
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.metrics import matthews_corrcoef
import numpy as np
np.random.seed(42)
import torch
import torch.nn as nn
from torch.utils.data import Dataset
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
dataset_path = "../datasets/mediaeval_fake_news_dataset/graphs/"
df, graph_dict, node_df = get_training_dataframe(dataset_path, two_class=False, get_node_df=True)
#node_df = node_df.set_index(['graph_id', 'node_id'])
print(graph_dict.keys()) # in this order in the df


dict_keys(['conspiracy_graphs', 'non_conspiracy_graphs', 'other_conspiracy_graphs'])


# Brainome

In [6]:
# for basic features

df.shape
df.to_csv("basic_features.csv", index=False)
# run btc -measureonly basic_features.csv -target label -ignorecolumns nodeID_list


for two class, brainome gives Best guess accuracy: 88.40%. This is just choosing 0 for everything. Got yellow. For two class, also got the same as choosing most common"

In [9]:
consp, non_consp, other_consp = graphs['conspiracy_graphs'], graphs['non_conspiracy_graphs'], graphs['other_conspiracy_graphs']
g2v1 = get_weighted_average_embeddings(consp, label=1)
g2v2 = get_weighted_average_embeddings(non_consp, label=2)
g2v3 = get_weighted_average_embeddings(other_consp, label=3)
g2v_dfs = [g2v1, g2v2, g2v3]

,weighted_g2v0,weighted_g2v1,weighted_g2v2,weighted_g2v3,weighted_g2v4,weighted_g2v5,weighted_g2v6,weighted_g2v7,weighted_g2v8,weighted_g2v9,...,weighted_g2v55,weighted_g2v56,weighted_g2v57,weighted_g2v58,weighted_g2v59,weighted_g2v60,weighted_g2v61,weighted_g2v62,weighted_g2v63,label
0,-0.135185,-0.048401,-0.039733,0.052115,-0.140880,0.107147,-0.059703,-0.047766,0.025264,0.053888,...,0.051684,0.111765,-0.000768,0.051560,0.172556,-0.056201,-0.206576,-0.131849,-0.032903,1
1,-0.166506,-0.058801,-0.061478,0.057558,-0.184593,0.148296,-0.057601,-0.060534,0.020224,0.068468,...,0.072657,0.124573,0.001599,0.080217,0.233222,-0.069000,-0.245228,-0.159604,-0.037272,1
2,-0.074059,-0.031809,-0.025210,0.028706,-0.074920,0.061906,-0.036715,-0.027512,0.012253,0.033411,...,0.034663,0.066087,-0.002857,0.034992,0.098433,-0.027912,-0.122374,-0.080374,-0.010138,1
3,-0.096855,-0.040148,-0.038347,0.037246,-0.092192,0.074869,-0.029323,-0.041718,0.010148,0.043166,...,0.046897,0.082165,0.003455,0.043655,0.126909,-0.047726,-0.144647,-0.093099,-0.017362,1
4,-0.078578,-0.033788,-0.031015,0.034652,-0.087756,0.071091,-0.030422,-0.028083,0.016498,0.028220,...,0.029060,0.067963,-0.002160,0.032116,0.111422,-0.027850,-0.119581,-0.078706,-0.010749,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,-0.119380,-0.044606,-0.038471,0.041389,-0.121399,0.099895,-0.046840,-0.043110,0.013045,0.045818,...,0.044321,0.091078,0.006572,0.052187,0.156923,-0.046507,-0.174894,-0.109014,-0.023418,1
266,-0.102998,-0.034455,-0.030975,0.034713,-0.103247,0.075003,-0.038463,-0.041934,0.014526,0.033481,...,0.036479,0.072301,0.004753,0.046622,0.120859,-0.039335,-0.137999,-0.088078,-0.022270,1
267,-0.147454,-0.052583,-0.057560,0.056190,-0.171025,0.146895,-0.072731,-0.048643,0.017665,0.060857,...,0.054055,0.116071,-0.007857,0.081087,0.213297,-0.068011,-0.221993,-0.164980,-0.022520,1
268,-0.005257,0.000199,0.000832,0.001242,-0.004764,0.002313,-0.001831,-0.000076,0.000840,0.001468,...,0.002257,0.000675,-0.000042,0.001870,0.002764,-0.002260,-0.006081,-0.002542,0.001515,1


In [15]:
all_g2v_embeddings = pd.concat(g2v_dfs)
all_g2v_embeddings.to_csv("g2v_weighted.csv", index=False)

# Using only Deepwalk
run deepwalk --input ../for_deepwalk.edgelist --format edgelist --output OUTFILE_NAME.txt --workers 4 --seed 42 --undirected false

In [17]:
composite_graph = clean_for_deepwalk(dataset_path, directed=True)
!deepwalk --input for_deepwalk.edgelist --format edgelist --output march20.txt --workers 4 --seed 42 --undirected false


### Nodes all have unique IDs

In [42]:
graphs['conspiracy_graphs'][0].nodes()

NodeView(('32115079', '449009614', '833469625', '499613745', '450210178', '182510449', '74719838'))

In [12]:
embeddings = pd.read_csv('deepwalk-master/march20.txt', names=['deepwalk' + str(i) for i in range(64)], sep=" ")
#set index (node labels) to string for consistency with NetworkX
as_string = embeddings.index.astype('string')
embeddings = embeddings.set_index(as_string)
embeddings.head(3)

,deepwalk0,deepwalk1,deepwalk2,deepwalk3,deepwalk4,deepwalk5,deepwalk6,deepwalk7,deepwalk8,deepwalk9,...,deepwalk54,deepwalk55,deepwalk56,deepwalk57,deepwalk58,deepwalk59,deepwalk60,deepwalk61,deepwalk62,deepwalk63
127166025,0.327048,-0.175018,0.224296,-0.304788,-0.222251,0.535962,-0.267580,0.059097,0.178404,-0.844267,...,-0.244400,-0.030048,-0.119165,0.218429,-0.567515,0.166872,-0.065369,-0.233754,-0.055578,0.034475
61721125,0.514014,-0.111906,-0.504094,0.304461,0.119720,0.193295,-0.145294,-0.607624,0.072884,0.312199,...,0.537953,-0.267592,-0.209925,0.365644,-0.741665,-0.070278,0.025231,-0.253768,0.119094,0.541069
127471447,0.320193,0.299958,0.126426,-0.828930,0.136684,0.066215,0.340867,-1.203018,-0.158095,0.751009,...,0.306866,-0.620950,1.211668,-0.151608,-0.585739,-0.606053,0.755599,-0.407707,0.263721,-0.653597


## Trying out an LSTM
### First, we need to create our dataset

In [35]:
class GraphDataset(Dataset):
    def __init__(self, df, embeddings, node_df, transform=None, target_transform=None):
        self.df = df[['label', 'nodeID_list']]
        self.embeddings = embeddings
        self.num_points = df.shape[0]
        self.labels = df['label']
    def __len__(self):
        return self.num_points
    def __getItem__(self, idx):
        #gonna have to transform data here
        return self.df.iloc[idx, :]

def x_transform(row, embeddings, node_df):
    """
    row:    row of the original df corresponding to the graph being transformed
    """
    nodes = row['nodeID_list']
    node_embeddings = embeddings[[(node in nodes) for node in embeddings.index]]

    graph_id = row['graph_id']
    node_info = node_df[(node_df['graph_id'] == graph_id) & (node_df['node_id'].isin(nodes))]
    all_features = pd.merge(right=node_embeddings, right_index=True, left=node_info, left_on='node_id')
    all_features = all_features.drop(columns=['graph_id', 'node_id'])
    return torch.tensor(all_features.values)


### Next, we create our RNN Model

In [166]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 0)
        hidden = self.i2h(combined)
        hidden = self.relu(hidden)
        output = self.i2o(combined)
        output = self.softmax(output)
        return hidden, output

    def initHidden(self):
        return torch.zeros(1, self.hidden_size).double()

### We define a loop for training our model

## using a basic logistic regression model to test what works

In [19]:
dw_feats = df[['label', 'nodeID_list']]
def get_deepwalk_mean(row):
    graph_nodes = row['nodeID_list']
    node_embed = embeddings[[(node in graph_nodes) for node in embeddings.index]]
    avg_embed = node_embed.mean(axis=0)
    return avg_embed

dw_feats[['dw_mean'+str(i) for i in range(64)]] = dw_feats.apply(get_deepwalk_mean, axis=1)
dw_feats.head(3)

KeyboardInterrupt: 

In [15]:
# get data first
dw_feats = dw_feats.drop('nodeID_list', axis=1)
X, y = np.array(dw_feats.drop('label', axis=1)), np.ravel(dw_feats[['label']])
kf = KFold(n_splits=5, random_state=42, shuffle=True)


In [16]:
scores = []
for train_idx, val_idx in kf.split(X):
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]
    model = LogisticRegression(random_state=42, class_weight='balanced', C=.0001)
    from sklearn import svm
    model = svm.SVC(kernel='rbf', class_weight="balanced", C=5)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    score = matthews_corrcoef(y_val, preds)
    #print(preds)
    scores.append(score)
scores

[0.33811562230497183,
 0.4170142973854589,
 0.36276128940856917,
 0.3848279938846305,
 0.3644198947839967]